**Resources list**
1. pyspark tutorial https://github.com/pydatasg/Pydata_meetup_Nov_16/blob/master/RF_modeling_2.3_business_weimin.ipynb
2. follow this tutorial: https://www.analyticsvidhya.com/blog/2016/10/spark-dataframe-and-operations/
3. and this one: https://www.dezyre.com/apache-spark-tutorial/pyspark-tutorial
4. data engineer trick: http://nadbordrozd.github.io/blog/2016/05/22/one-weird-trick-that-will-fix-your-pyspark-schemas/
5. git http://rogerdudler.github.io/git-guide/

# Get data
## Downloand data from s3
version 1.0

In [2]:
! cd /home/ubuntu/s3/flight_1_5
! aws s3 sync s3://flight.price/flight_1_5 . 

download: s3://flight.price/flight_1_5/flight_1_5_price_2017-04-14.zip to ./flight_1_5_price_2017-04-14.zip
download: s3://flight.price/flight_1_5/flight_1_5_price_2017-04-04.zip to ./flight_1_5_price_2017-04-04.zip
download: s3://flight.price/flight_1_5/flight_1_5_price_2017-04-15.zip to ./flight_1_5_price_2017-04-15.zip
download: s3://flight.price/flight_1_5/flight_1_5_price_2017-04-18.zip to ./flight_1_5_price_2017-04-18.zip
download: s3://flight.price/flight_1_5/flight_1_5_price_2017-04-16.zip to ./flight_1_5_price_2017-04-16.zip
download: s3://flight.price/flight_1_5/flight_1_5_price_2017-04-19.zip to ./flight_1_5_price_2017-04-19.zip
download: s3://flight.price/flight_1_5/flight_1_5_price_2017-04-20.zip to ./flight_1_5_price_2017-04-20.zip
download: s3://flight.price/flight_1_5/flight_1_5_price_2017-04-21.zip to ./flight_1_5_price_2017-04-21.zip
download: s3://flight.price/flight_1_5/flight_1_5_price_2017-04-22.zip to ./flight_1_5_price_2017-04-22.zip
download: s3://flight.price/

version 1.1

In [13]:
! aws s3 ls s3://flight.price.11/flight_1_5 --recursive
! aws s3 sync s3://flight.price.11/flight_1_5 .

2017-05-11 12:21:21   51976702 flight_1_5/flight_1_5_price_2017-05-11.zip
2017-05-12 11:41:26  488154487 flight_1_5/flight_1_5_price_2017-05-12.zip
2017-05-13 11:12:53  488920458 flight_1_5/flight_1_5_price_2017-05-13.zip
2017-05-14 23:50:24  489340830 flight_1_5/flight_1_5_price_2017-05-14.zip
download: s3://flight.price.11/flight_1_5/flight_1_5_price_2017-05-11.zip to ./flight_1_5_price_2017-05-11.zip
download: s3://flight.price.11/flight_1_5/flight_1_5_price_2017-05-14.zip to ./flight_1_5_price_2017-05-14.zip
download: s3://flight.price.11/flight_1_5/flight_1_5_price_2017-05-13.zip to ./flight_1_5_price_2017-05-13.zip
download: s3://flight.price.11/flight_1_5/flight_1_5_price_2017-05-12.zip to ./flight_1_5_price_2017-05-12.zip


In [17]:
! du -h

4.0K	./spark-warehouse
1.5G	./extracted/final_results
1.6G	./extracted
4.5G	.


## Extract
Note: version 1.1 from 2017-05-11 onwards

In [20]:
import zipfile
import os

# dir_in = 'C:\\s3\\20170503\\flight_1_6' # sydney to shanghai
# dir_in = 'C:\\s3\\20170503\\flight_5_1' # beijing to sydney
# dir_in = 'C:\\s3\\20170503'
dir_in = '/home/ubuntu/s3/flight_1_5'

# dir_out = 'C:\\s3\\20170503_extracted\\flight_1_6'
dir_out = '/home/ubuntu/s3/flight_1_5/extracted'
extension = ".zip"

os.chdir(dir_in) # change directory from working dir to dir with files

for subdir, dirs, files in os.walk(dir_in):
    for item in files:
        if item.endswith(extension): # check for ".zip" extension
            file_name = os.path.join(subdir, item)
#             file_name = os.path.abspath(item) # get full path of files
            zip_ref = zipfile.ZipFile(file_name) # create zipfile object
            zip_ref.extractall(dir_out) # extract file to dir
            zip_ref.close() # close file  

Move files in final_results to the main folder and delete the folder

In [34]:
# ! cd /home/ubuntu/s3/flight_1_5/extracted
# ! mv /home/ubuntu/s3/flight_1_5/extracted/final_results/*.txt .
! find /home/ubuntu/s3/flight_1_5/extracted/final_results -name '*.txt' -exec mv {} /home/ubuntu/s3/flight_1_5/extracted \;
# -exec runs any command,  {} inserts the filename found, \; marks the end of the exec command.
! rmdir /home/ubuntu/s3/flight_1_5/extracted/final_results

http://stackoverflow.com/questions/9157138/recursively-counting-files-in-a-linux-directory
* -type f to include only files.  
* | (and not ¦) redirects find command's standard output to wc command's standard input.  
* wc (short for word count) counts newlines, words and bytes on its input (docs).  
* -l to count just newlines.
* You can also remove the -type f to include directories (and symlinks) in the count.
* It's possible this command will overcount if filenames can contain newline characters.

In [36]:
! find /home/ubuntu/s3/flight_1_5/extracted -type f | wc -l
! du -sh

26261
5.9G	.


# Launch spark

In [1]:
import h2o
# import pysparkling
import os
import sys
from pyspark.sql import SparkSession
from IPython.display import display
from pyspark.sql.functions import regexp_extract, col, split, udf, trim, when, from_unixtime, unix_timestamp, minute, hour, datediff
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, StringType, BooleanType
# from pyspark.sql.types import *
import datetime

from pyspark import SparkContext
sc = SparkContext()

# # # spark_home = os.environ.get('SPARK_HOME', None)
# spark_home = "C:\spark-2.1.0-bin-hadoop2.7\spark-2.1.0-bin-hadoop2.7"

# if not spark_home:

#     raise ValueError('SPARK_HOME environment variable is not set')

# sys.path.insert(0, os.path.join(spark_home, 'python'))

# sys.path.insert(0, os.path.join(spark_home, 'C:\spark-2.1.0-bin-hadoop2.7\spark-2.1.0-bin-hadoop2.7\python\lib\py4j-0.10.4-src.zip')) ## may need to adjust on your system depending on which Spark version you're using and where you installed it.

# exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
        
spark.version

In [1]:
# hc= H2OContext(sc).start()

In [107]:
# h2o not working yet
# # Start H2O Context
# from pysparkling import *
# sc
# hc= H2OContext(sc).start()

## Read in json files and aggregate into a dataframe

In [108]:
#!/usr/bin/env python3
# Copyright (C) 2015-2025 Wang,Jing   <jingwangian@gmail.com>
#
# This file is part of Flight Inforation Query System (fiqs)
#
# fiqs is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# fiqs is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with fiqs.  If not, see <http://www.gnu.org/licenses/>.

import argparse
import json
from pyspark.sql import SparkSession

# def spark_write(spark,jsonl_file_name,parquet_file_name):
#     df = spark.read.json(jsonl_file_name)
#     df.write.parquet(parquet_file_name)
    
# def spark_print_json(spark,json_file_name):
#     d = read_from_json_file(json_file_name)
    
#     jsonl_file_name = json_file_name.replace('.json','.jsonl')
    
#     with open(jsonl_file_name,'w') as f:
#         json.dump(d, f)
    
#     df = spark.read.json(jsonl_file_name)
    
#     df.createGlobalTempView("flight")
    
#     spark.sql("SELECT fromCity,toCity,flight_leg1.departureTime, flight_leg1.departureLocation.airportLongName FROM global_temp.flight").show()
    
def read_from_json_file(filename):
    with open(filename,'r') as f:
        d = json.load(f)
        
    return d

# def pq_to_json_file(spark, pq_file_name, json_file_name):
#     df = spark.read.load(pq_file_name)
    
#     df.write.json(json_file_name)
    
# def read_data_from_pq_file(spark,filename):
#     df = spark.read.load(filename)
    
#     df.select('from_city_name','to_city_name').show()
#     #spark.sql("SELECT from_city_name,to_city_name FROM global_temp.flight").show()
    
# def main():
#     full_path_pq_file_name = "test/t.parquet"

#     spark = SparkSession \
#         .builder \
#         .appName("Python Spark SQL basic example") \
#         .config("spark.some.config.option", "some-value") \
#         .getOrCreate()
            
#     parser = argparse.ArgumentParser()
    
#     parser.add_argument('cmd',help='[json,pq,json2pq,jsonprint]')
#     parser.add_argument('--json',help='json file namejson,pq]')
#     parser.add_argument('--pq',help='parquet file name')
    
#     args = parser.parse_args()
    
#     cmd_ind = args.cmd
#     json_file_name = args.json
#     pq_file_name = args.pq
    
#     if cmd_ind == 'json2pq':
#         if json_file_name is None:
#             print("Please set the json file name by --json")
#             quit(0)
            
#         if pq_file_name is None:
#             print("Please set the pqrquet file name by --pq")
#             quit(0)
#         d = read_from_json_file(json_file_name)
        
#         jsonl_file_name = json_file_name.replace('.json','.jsonl')
        
#         with open(jsonl_file_name,'w') as f:
#             json.dump(d, f)
            
#         spark_write(spark, jsonl_file_name, pq_file_name)
#     elif cmd_ind == 'pq':
#         if pq_file_name is None:
#             print("Please set the pqrquet file name by --pq")
#             quit(0)
#         read_data_from_pq_file(spark,full_path_pq_file_name)
#     elif cmd_ind == 'pq2json':
#         if json_file_name is None:
#             print("Please set the json file name by --json")
#             quit(0)
            
#         if pq_file_name is None:
#             print("Please set the pqrquet file name by --pq")
#             quit(0)
            
#         pq_to_json_file(spark,pq_file_name,json_file_name)
        
#     elif cmd_ind== 'jsonprint':
#         if json_file_name is None:
#             print("Please set the json file name by --json")
#             quit(0)
            
#         spark_print_json(spark,json_file_name)
#     else:
#         print('invalid cmd command')
        
#     spark.stop()
    
    
# if __name__== '__main__':
#     main()
    
    
# spark-submit sp.py jsonprint --json final_flight_result_format_v1.1.json


# json_file_name = "D:\\Data Science\\pySpark\\test_schema\\final_flight_result_format_v1.1.json"
# jsonl_file_name = "C:\\s3\\20170503_jsonl\\flight_1_6.jsonl"
jsonl_file_name = "C:\\s3\\a3.json"

# d = read_from_json_file(json_file_name)

# jsonl_file_name = json_file_name.replace('.json','.jsonl')

# with open(jsonl_file_name,'w') as f:
#     json.dump(d, f)

# flight = spark.read.json(jsonl_file_name)

# df.createGlobalTempView("flight")

# spark.sql("SELECT fromCity,toCity,flight_leg1.departureTime, flight_leg1.departureLocation.airportLongName FROM global_temp.flight").show()
  
# flight.printSchema()


In [109]:
# flight.printSchema()
# flight.drop('airline_codes').repartition(1).write.format('com.databricks.spark.csv').save('C:\\s3\\20170503_jsonl\\flight_1_6.csv', header=True)

In [110]:
# flight.write.parquet("C:\\s3\\20170503_jsonl\\flight.parquet")

In [2]:
# flight = spark.read.parquet("C:\\s3\\20170503_jsonl\\flight.parquet")
flight = spark.read.parquet("/home/ubuntu/parquet/flight.parquet")

In [3]:
flight.count()

1862553

In [4]:
flight.printSchema()
flight.show(2,truncate=False)
display(flight.count())

root
 |-- airline_code: string (nullable = true)
 |-- airline_codes: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- arr_time: string (nullable = true)
 |-- check_bag_inc: boolean (nullable = true)
 |-- company: string (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- flight_code: string (nullable = true)
 |-- flight_number: string (nullable = true)
 |-- from_city_name: string (nullable = true)
 |-- id: string (nullable = true)
 |-- index: long (nullable = true)
 |-- plane: string (nullable = true)
 |-- power: boolean (nullable = true)
 |-- price: double (nullable = true)
 |-- price_code: string (nullable = true)
 |-- search_date: string (nullable = true)
 |-- span_days: long (nullable = true)
 |-- start_date: string (nullable = true)
 |-- stay_days: string (nullable = true)
 |-- stop: long (nullable = true)
 |-- stop_info: string (nullable = true)
 |-- table_name: string (nullable = true)
 |-- task_id: s

1862553

In [113]:
# path_to_input = "D:\Data Science\pySpark\original_json"
# original_data = spark.read.json(sc.wholeTextFiles(path_to_input).values())

# original_data = sqlContext.read.json("D:\\Data Science\\pySpark\\original_json\\original_flight_info.txt")
# original_data.printSchema()


In [114]:
# import json
# from pprint import pprint

# # with open("D:\\Data Science\\pySpark\\original_json\\original_flight_info.txt") as data_file:    
# #     data = json.load(data_file)

# # pprint(data)
# # # data.head()

# df = spark.read.json("D:\\Data Science\\Flight v2\\python\\final_flight_result_format_v1.1.json")
# # Displays the content of the DataFrame to stdout
# df.show()




In [115]:
# flight = sc.textFile("C:\\s3_temp\\summary\\flat_table.csv").map(lambda line: (line.split(',')[5], line.split(',')[10])).collect()
# flight = sc.textFile("C:\\s3_temp\\summary\\flat_table.csv")
# flight = spark.read.csv("C:\\s3_temp\\summary\\flat_table.csv", header=True, mode="DROPMALFORMED")

# flight.cache()

In [116]:
# flight.repartition(1).write.format('com.databricks.spark.csv').save('D://Data Science//pySpark//test2.csv', header=True)


In [5]:
flight.first()

Row(airline_code='QF', airline_codes=['QF'], arr_time='2017-04-05T19:20:00.000+08:00', check_bag_inc=False, company='Qantas Airways', dep_time='2017-04-05T11:00:00.000+10:00', duration='10h20m', flight_code='QF301', flight_number='301', from_city_name='sydney', id=None, index=12, plane='AIRBUS INDUSTRIE A330-200', power=False, price=1178.42, price_code='AUD', search_date='2017-04-04', span_days=0, start_date='2017-04-05', stay_days='0', stop=0, stop_info='', table_name='flight_1_6_price', task_id='901', ticket_left=9, to_city_name='shanghai', trip='1', version='1.0', video=True, wifi=False)

In [9]:
# flight.rdd.takeSample(False, 3, 1234)

In [6]:
flight.dtypes

[('airline_code', 'string'),
 ('airline_codes', 'array<string>'),
 ('arr_time', 'string'),
 ('check_bag_inc', 'boolean'),
 ('company', 'string'),
 ('dep_time', 'string'),
 ('duration', 'string'),
 ('flight_code', 'string'),
 ('flight_number', 'string'),
 ('from_city_name', 'string'),
 ('id', 'string'),
 ('index', 'bigint'),
 ('plane', 'string'),
 ('power', 'boolean'),
 ('price', 'double'),
 ('price_code', 'string'),
 ('search_date', 'string'),
 ('span_days', 'bigint'),
 ('start_date', 'string'),
 ('stay_days', 'string'),
 ('stop', 'bigint'),
 ('stop_info', 'string'),
 ('table_name', 'string'),
 ('task_id', 'string'),
 ('ticket_left', 'bigint'),
 ('to_city_name', 'string'),
 ('trip', 'string'),
 ('version', 'string'),
 ('video', 'boolean'),
 ('wifi', 'boolean')]

In [7]:
# from pyspark.sql.functions import year, month, dayofmonth, hour, minute, weekofyear, crosstab
# flight2.select(year("arr_time").alias('year'), 
# from pyspark.sql.functions import regexp_extract, col, split
# # import re

# from pyspark.sql.functions import udf
# from pyspark.sql.types import IntegerType, StringType, BooleanType

def toBool(aString):
    if aString.lower() == 'true':
        return True
    else: 
        return False
 
udfToBool = udf(toBool, BooleanType())


def groupTicketLeft(aString):
    if aString == '0':
        return 'plenty'
    else: 
        return aString
 
udfGroupTicketLeft = udf(groupTicketLeft, StringType())

# def getMinutes(aString):
#     return minute(unix_timestamp(aString, "HH'h'mm'm'").cast("timestamp"))

# udfGetMinutes = udf(getMinutes, IntegerType())

# def getHours(x):
#   return re.match('([0-9]+(?=h))', x)
# getHours("14h")
# len(flight.columns)
# flight.columns
# flight.dtypes
flight2 = (flight.withColumn('search_date', flight.search_date.cast('timestamp'))
#           .withColumn('search_date_y', flight.search_date_y.cast('timestamp'))
#           .withColumnRenamed('search_date_y', 'search_date')
          .withColumn('stay_days', flight.stay_days.cast('int'))                                
           #create local time first
           .withColumn('dep_time_local', flight.dep_time.substr(1, 23).cast('timestamp'))                      
          .withColumn('dep_time', flight.dep_time.cast('timestamp'))
           #create local time first
          .withColumn('arr_time_local', flight.arr_time.substr(1, 23).cast('timestamp'))                      
          .withColumn('arr_time', flight.arr_time.cast('timestamp'))
           #duration           
           .withColumn('duration_h',split(flight.duration,'h').getItem(0))
           .withColumn('duration_m',split(flight.duration,'h').getItem(1))
           #stop info
           .withColumn('stop_info1',split(flight.stop_info,';').getItem(0))
           .withColumn('stop_info2',split(flight.stop_info,';').getItem(1))
#            .withColumn('druation_hours', 
#                        flight.selectExpr("duration", "regexp_extract(duration,'([0-9]+(?=h))', 1) as duration_hours")
#                        .duration_hours
#                        .cast('int'))
          .withColumn('start_date', flight.start_date.cast('date'))           
          .withColumn('price', flight.price.cast('double'))          
           # to correct the name
          .withColumnRenamed('check_bag_inc', 'check_bag_not_inc')
#           .withColumn('check_bag_not_inc', udfToBool(flight.check_bag_inc))
          .withColumn('power', flight.power.cast('boolean'))           
          .withColumn('video', flight.video.cast('boolean'))
          .withColumn('wifi', flight.wifi.cast('boolean'))           
          .withColumn('stop', flight.stop.cast('int')) 
          .withColumn('span_days', flight.span_days.cast('int'))           
           .withColumn('ticket_left', udfGroupTicketLeft(flight.ticket_left))
           .drop('search_date_x', 'check_bag_inc')
          )

flight2 = (flight2.withColumn('stop_loc1',split(flight2.stop_info1,':').getItem(0))
           .withColumn('stop_duration1',split(flight2.stop_info1, ':').getItem(1))
           .withColumn('stop_loc2',split(flight2.stop_info2,':').getItem(0))
           .withColumn('stop_duration2',split(flight2.stop_info2, ':').getItem(1)))

flight2 = (flight2.withColumn('stop_duration_h1',split(flight2.stop_duration1,'h').getItem(0))
           .withColumn('stop_duration_m1',split(flight2.stop_duration1,'h').getItem(1))
           .withColumn('stop_duration_h2',split(flight2.stop_duration2,'h').getItem(0))
           .withColumn('stop_duration_m2',split(flight2.stop_duration2,'h').getItem(1)))
# .withColumn('arr_time_zone', flight.arr_time.substr(24, 6))

In [121]:
# Define udf

def getMinutes(hString, minString):
    if (hString is not None) & (minString is not None): return int(hString) * 60 + int(minString[:-1])
    else: return None
#     if score >= 80: return 'A'
#     elif score >= 60: return 'B'
#     elif score >= 35: return 'C'
#     else: return 'D'
 
udfGetMinutes = udf(getMinutes, IntegerType())
flight2 = (flight2.withColumn("duration_minutes", udfGetMinutes("duration_h", "duration_m"))
                    .withColumn("stop1_minutes", udfGetMinutes("stop_duration_h1", "stop_duration_m1"))
                   .withColumn("stop2_minutes", udfGetMinutes("stop_duration_h2", "stop_duration_m2")))
                 
flight2.show(2)

+------------+-------------+--------------------+-----------------+----------------+--------------------+--------+-----------+-------------+--------------+----+-----+--------------------+-----+-------+----------+--------------------+---------+----------+---------+----+-----------------+----------------+-------+-----------+------------+----+-------+-----+-----+--------------------+--------------------+----------+----------+-----------------+----------+----------+--------------+---------+--------------+----------------+----------------+----------------+----------------+----------------+-------------+-------------+
|airline_code|airline_codes|            arr_time|check_bag_not_inc|         company|            dep_time|duration|flight_code|flight_number|from_city_name|  id|index|               plane|power|  price|price_code|         search_date|span_days|start_date|stay_days|stop|        stop_info|      table_name|task_id|ticket_left|to_city_name|trip|version|video| wifi|      dep_time_loc

In [13]:
# flight2.groupby('search_date', 'from_city_name', 'to_city_name', 'trip').count().show()
# flight2.groupby('table_name').count().show()
flight2.groupby('stop_info', 
                'stop_info1', 'stop_loc1', 
                'stop_duration1', 'stop_duration_h1', 'stop_duration_m1',
                'stop_info2', 'stop_loc2', 
                'stop_duration2',  'stop_duration_h2', 'stop_duration_m2').count().show(500, truncate=False)
# flight2.groupby('stop_duration1').count().show()

Py4JJavaError: An error occurred while calling o286.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8.0 failed 1 times, most recent failure: Lost task 0.0 in stage 8.0 (TID 6, localhost, executor driver): java.lang.RuntimeException: file:/home/ubuntu/flight.parquet/part-00008-f2da479a-50b6-4ead-9188-97fc7359de20.snappy.parquet is not a Parquet file. expected magic number at tail [80, 65, 82, 49] but found [0, 33, 32, 33]
	at org.apache.parquet.hadoop.ParquetFileReader.readFooter(ParquetFileReader.java:423)
	at org.apache.parquet.hadoop.ParquetFileReader.readFooter(ParquetFileReader.java:386)
	at org.apache.spark.sql.execution.datasources.parquet.SpecificParquetRecordReaderBase.initialize(SpecificParquetRecordReaderBase.java:107)
	at org.apache.spark.sql.execution.datasources.parquet.VectorizedParquetRecordReader.initialize(VectorizedParquetRecordReader.java:109)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$$an

In [14]:
flight2.dtypes

[('airline_code', 'string'),
 ('airline_codes', 'array<string>'),
 ('arr_time', 'timestamp'),
 ('check_bag_not_inc', 'boolean'),
 ('company', 'string'),
 ('dep_time', 'timestamp'),
 ('duration', 'string'),
 ('flight_code', 'string'),
 ('flight_number', 'string'),
 ('from_city_name', 'string'),
 ('id', 'string'),
 ('index', 'bigint'),
 ('plane', 'string'),
 ('power', 'boolean'),
 ('price', 'double'),
 ('price_code', 'string'),
 ('search_date', 'timestamp'),
 ('span_days', 'int'),
 ('start_date', 'date'),
 ('stay_days', 'int'),
 ('stop', 'int'),
 ('stop_info', 'string'),
 ('table_name', 'string'),
 ('task_id', 'string'),
 ('ticket_left', 'string'),
 ('to_city_name', 'string'),
 ('trip', 'string'),
 ('version', 'string'),
 ('video', 'boolean'),
 ('wifi', 'boolean'),
 ('dep_time_local', 'timestamp'),
 ('arr_time_local', 'timestamp'),
 ('duration_h', 'string'),
 ('duration_m', 'string'),
 ('stop_info1', 'string'),
 ('stop_info2', 'string'),
 ('stop_loc1', 'string'),
 ('stop_duration1', 'str

In [ ]:
# flight2 = flight2.drop('stop_duration_h1', 'stop_duration_m1', 'stop_duration_h2', 'stop_duration_m2',
#                       'stop_duration1', 'stop_duration2',
#                       'duration_h', 'duration_m', 
#                        'stop_info', 'stop_info1', 'stop_info2')
# display(flight2.show(2))
# display(flight2.dtypes)

In [15]:
flight2.show(2)

+------------+-------------+--------------------+-----------------+----------------+--------------------+--------+-----------+-------------+--------------+----+-----+--------------------+-----+-------+----------+--------------------+---------+----------+---------+----+-----------------+----------------+-------+-----------+------------+----+-------+-----+-----+--------------------+--------------------+----------+----------+-----------------+----------+----------+--------------+---------+--------------+----------------+----------------+----------------+----------------+
|airline_code|airline_codes|            arr_time|check_bag_not_inc|         company|            dep_time|duration|flight_code|flight_number|from_city_name|  id|index|               plane|power|  price|price_code|         search_date|span_days|start_date|stay_days|stop|        stop_info|      table_name|task_id|ticket_left|to_city_name|trip|version|video| wifi|      dep_time_local|      arr_time_local|duration_h|duration_m

In [16]:
def groupTime(dt):
    h = dt.hour
    if h < 5 : return 'early morning'    
    elif h < 12 : return 'morning'
    elif h < 18 : return 'afternoon'    
    else: return 'evening'
#     if h >= 18 or h < 7 : return 'night'    
#     elif 7 <= h < 12 : return 'morning'
#     elif 12 <= h < 18 : return 'afternoon'    
#     else: return 'error'
    
    
udfGroupTime = udf(groupTime, StringType())
flight2 = flight2.withColumn("arr_time_group", udfGroupTime("arr_time_local")) \
                .withColumn("dep_time_group", udfGroupTime("dep_time_local"))              
flight2.cache()

DataFrame[airline_code: string, airline_codes: array<string>, arr_time: timestamp, check_bag_not_inc: boolean, company: string, dep_time: timestamp, duration: string, flight_code: string, flight_number: string, from_city_name: string, id: string, index: bigint, plane: string, power: boolean, price: double, price_code: string, search_date: timestamp, span_days: int, start_date: date, stay_days: int, stop: int, stop_info: string, table_name: string, task_id: string, ticket_left: string, to_city_name: string, trip: string, version: string, video: boolean, wifi: boolean, dep_time_local: timestamp, arr_time_local: timestamp, duration_h: string, duration_m: string, stop_info1: string, stop_info2: string, stop_loc1: string, stop_duration1: string, stop_loc2: string, stop_duration2: string, stop_duration_h1: string, stop_duration_m1: string, stop_duration_h2: string, stop_duration_m2: string, arr_time_group: string, dep_time_group: string]

In [ ]:
import pyspark.sql.functions as F

# flight2.select("dep_time_local", "dep_time_group", "arr_time_local", "arr_time_group").show(10)
#flight2.describe(['price']).show()# groupBy("dep_time_group", "arr_time_group")
# flight2.freqItems(flight2.trip)
# flight2.groupby(flight2.trip, flight2.stay_days, flight.dep_time_group).agg(func.mean('price')).show()
# flight2.groupby(flight2.trip, flight2.dep_time_group).agg(func.mean('price')).orderBy(['trip', 'avg(price)'], ascending=[1, 0]).show()
# flight2.groupby(flight2.trip, flight2.company).agg(F.mean('price'), F.stddev('price'), F.max('price'), F.min('price')).orderBy(['trip', 'stddev_samp(price)'], ascending=[1, 0]).show(10000)
flight2.filter(flight2.price <= 0).count() / flight2.count()

# drop rows with zero price
flight2 = flight2.filter(flight2.price > 0)

In [ ]:
# duration_test.select(trim(col("duration"))).show()
# df
# .withColumn('Created-formatted',when((df.Created.isNull() | (df.Created == '')) ,'0')
# .otherwise(unix_timestamp(df.Created,'yyyy-MM-dd')))

duration_test = flight2.select("stop_duration1")
duration_test.show()

duration_test.withColumn('duration_h', when(duration_test.stop_duration1.isNull(), None)
                          .otherwise(hour(unix_timestamp(duration_test.stop_duration1,"HH'h'mm'm'").cast("timestamp")))).show(20)          
    
#     .withColumn('duration_m', minute(unix_timestamp(duration_test.duration,"HH'h'mm'm'").cast("timestamp")))).show(2)


# duration_test.withColumn('duration2',when((duration_test.duration.isNull() | (duration_test.duration == '')) , 0)\
#   .otherwise(unix_timestamp(duration_test.duration,"HH'h'mm'm'").cast("timestamp"))).show(2)

In [129]:
# from datetime import date

# import holidays

# us_holidays = holidays.UnitedStates()  # or holidays.US()

# date(2015, 1, 1) in us_holidays  # True
# date(2020, 12, 25) in us_holidays  # False

# # print(us_holidays)


# for date, name in sorted(holidays.AU(state='NSW', years=2017).items()):
#      print(date, name)
        
# for date, name in sorted(holidays.NZ(state='AUK', years=2017).items()):
#      print(date, name)

In [130]:


# # The Holiday class will also recognize strings of any format
# # and int/float representing a Unix timestamp
# '2014-01-01' in us_holidays  # True
# '1/1/2014' in us_holidays    # True
# 1388597445 in us_holidays    # True

# us_holidays.get('2014-01-01')  # "New Year's Day"

# # Easily create custom Holiday objects with your own dates instead
# # of using the pre-defined countries/states/provinces available
# custom_holidays = holidays.HolidayBase()
# # Append custom holiday dates by passing:
# # 1) a dict with date/name key/value pairs,
# custom_holidays.append({"2015-01-01": "New Year's Day"})
# # 2) a list of dates (in any format: date, datetime, string, integer),
# custom_holidays.append(['2015-07-01', '07/04/2015'])
# # 3) a single date item
# custom_holidays.append(date(2015, 12, 25))

# date(2015, 1, 1) in custom_holidays  # True
# date(2015, 1, 2) in custom_holidays  # False
# '12/25/2015' in custom_holidays      # True

# # For more complex logic like 4th Monday of January, you can inherit the
# # HolidayBase class and define your own _populate(year) method. See below
# # documentation for examples.

In [131]:
# from datetime import date
# from workalendar.europe import France
# cal = France()
# cal.holidays(2017)


In [132]:
# flight2.show(2)
flight3 = flight2


def getWeekday(dt):
    return dt.weekday()
    
def getWeeknumber(dt):
    return dt.isocalendar()[1]

    
udfGetWeekday = udf(getWeekday, StringType())
udfGetWeeknumber = udf(getWeeknumber, IntegerType())

flight3 = flight3.withColumn("dep_weekday", udfGetWeekday("start_date")) \
                .withColumn("dep_weeknum", udfGetWeeknumber("start_date")) \
                .withColumn("lead_time", datediff(flight3.start_date, flight3.search_date)) 
#                 .drop('start_date', 'search_date')

flight3.show(2)
display(flight2.count(), flight3.count())
flight3.describe('lead_time').show()

+------------+-------------+--------------------+-----------------+----------------+--------------------+--------+-----------+-------------+--------------+----+-----+--------------------+-----+-------+----------+--------------------+---------+----------+---------+----+-----------------+----------------+-------+-----------+------------+----+-------+-----+-----+--------------------+--------------------+----------+----------+-----------------+----------+----------+--------------+---------+--------------+----------------+----------------+----------------+----------------+----------------+-------------+-------------+--------------+--------------+-----------+-----------+---------+
|airline_code|airline_codes|            arr_time|check_bag_not_inc|         company|            dep_time|duration|flight_code|flight_number|from_city_name|  id|index|               plane|power|  price|price_code|         search_date|span_days|start_date|stay_days|stop|        stop_info|      table_name|task_id|tick

1835752

1835752

+-------+------------------+
|summary|         lead_time|
+-------+------------------+
|  count|           1835752|
|   mean| 90.30425950782023|
| stddev|51.508851575859815|
|    min|                 1|
|    max|               180|
+-------+------------------+



In [133]:
flight3.show(5)
flight3.dtypes

+------------+-------------+--------------------+-----------------+--------------------+--------------------+--------+-----------+-------------+--------------+----+-----+--------------------+-----+-------+----------+--------------------+---------+----------+---------+----+--------------------+----------------+-------+-----------+------------+----+-------+-----+-----+--------------------+--------------------+----------+----------+--------------------+----------+--------------+--------------+---------+--------------+----------------+----------------+----------------+----------------+----------------+-------------+-------------+--------------+--------------+-----------+-----------+---------+
|airline_code|airline_codes|            arr_time|check_bag_not_inc|             company|            dep_time|duration|flight_code|flight_number|from_city_name|  id|index|               plane|power|  price|price_code|         search_date|span_days|start_date|stay_days|stop|           stop_info|      ta

[('airline_code', 'string'),
 ('airline_codes', 'array<string>'),
 ('arr_time', 'timestamp'),
 ('check_bag_not_inc', 'boolean'),
 ('company', 'string'),
 ('dep_time', 'timestamp'),
 ('duration', 'string'),
 ('flight_code', 'string'),
 ('flight_number', 'string'),
 ('from_city_name', 'string'),
 ('id', 'string'),
 ('index', 'bigint'),
 ('plane', 'string'),
 ('power', 'boolean'),
 ('price', 'double'),
 ('price_code', 'string'),
 ('search_date', 'timestamp'),
 ('span_days', 'int'),
 ('start_date', 'date'),
 ('stay_days', 'int'),
 ('stop', 'int'),
 ('stop_info', 'string'),
 ('table_name', 'string'),
 ('task_id', 'string'),
 ('ticket_left', 'string'),
 ('to_city_name', 'string'),
 ('trip', 'string'),
 ('version', 'string'),
 ('video', 'boolean'),
 ('wifi', 'boolean'),
 ('dep_time_local', 'timestamp'),
 ('arr_time_local', 'timestamp'),
 ('duration_h', 'string'),
 ('duration_m', 'string'),
 ('stop_info1', 'string'),
 ('stop_info2', 'string'),
 ('stop_loc1', 'string'),
 ('stop_duration1', 'str

get distance to nearest holiday: http://stackoverflow.com/questions/40752378/spark-sql-distance-to-nearest-holiday
    

In [134]:


# datetime.datetime('2017-09-01')
# date1 = datetime.datetime.strptime("2015-01-30", "%Y-%m-%d").strftime("%d-%m-%Y")
# print(date1)
# start_date_test = flight3.select("start_date").distinct()
flight3 = flight3.withColumn("start_date_str", flight3.start_date.cast('string'))

holidays = ['2017-01-01', '2017-01-02', '2017-01-27', '2017-01-28',
            '2017-01-29', '2017-01-30', '2017-01-31', '2017-02-01', '2017-02-02',
            '2017-04-02', '2017-04-03', '2017-04-04',
            '2017-05-01', '2017-05-28', '2017-05-29', '2017-05-30',
            '2017-10-01', '2017-10-02', '2017-10-03', '2017-10-04', '217-10-05', '2017-10-06']
 

index = spark.sparkContext.broadcast(sorted(holidays))

def last_holiday(date):
    last_holiday = index.value[0]
    for next_holiday in index.value:
        if next_holiday >= date:
            break
        last_holiday = next_holiday
    if last_holiday > date:
        last_holiday = None
    if next_holiday < date:
        next_holiday = None        
    return last_holiday

def next_holiday(date):
    last_holiday = index.value[0]
    for next_holiday in index.value:
        if next_holiday >= date:
            break
        last_holiday = next_holiday
    if last_holiday > date:
        last_holiday = None
    if next_holiday < date:
        next_holiday = None        
    return next_holiday


# return_type = StructType([StructField('last_holiday', StringType()), StructField('next_holiday', StringType())])

last_holiday_udf = udf(last_holiday, StringType())
next_holiday_udf = udf(next_holiday, StringType())

flight4 = flight3.withColumn('last_holiday', last_holiday_udf('start_date_str'))
flight4 = flight4.withColumn('next_holiday', next_holiday_udf('start_date_str'))
flight4 = flight4.withColumn('days_to_last_holiday', datediff('last_holiday', 'start_date_str'))
flight4 = flight4.withColumn('days_to_next_holiday', datediff('next_holiday', 'start_date_str'))

In [135]:
(flight4
    .select('days_to_last_holiday', 'days_to_next_holiday', 'last_holiday', 'next_holiday', 'start_date')    
    .distinct().show())

+--------------------+--------------------+------------+------------+----------+
|days_to_last_holiday|days_to_next_holiday|last_holiday|next_holiday|start_date|
+--------------------+--------------------+------------+------------+----------+
|                 -40|                  84|  2017-05-30|  2017-10-01|2017-07-09|
|                 -17|                  10|  2017-04-04|  2017-05-01|2017-04-21|
|                  -8|                  19|  2017-04-04|  2017-05-01|2017-04-12|
|                 -34|                  90|  2017-05-30|  2017-10-01|2017-07-03|
|                 -22|                 102|  2017-05-30|  2017-10-01|2017-06-21|
|                 -70|                  54|  2017-05-30|  2017-10-01|2017-08-08|
|                 -75|                  49|  2017-05-30|  2017-10-01|2017-08-13|
|                 -28|                  96|  2017-05-30|  2017-10-01|2017-06-27|
|                 -16|                  11|  2017-04-04|  2017-05-01|2017-04-20|
|                 -12|      

In [218]:
# flight4.show()
#'table_name', 'trip', 'stay_days'
# flight5 = (flight4.filter((flight4.trip==1) & (flight4.start_date=='2017-10-01') & (flight4.company=='AirAsiaX'))
#          .select('start_date', 'company', 'dep_time_local', 'stop_info', 'duration', 'search_date', 'price')
#          .sort(F.desc('start_date'), 'company', 'dep_time_local', 'stop_info', 'duration', 'search_date'))       
         
flight5 = flight4.sort('table_name', 'trip', 'stay_days', 
                       'start_date', 'company', 'dep_time_local',
                       'stop_info', 'duration', 'search_date')       
    
byVar = ['table_name', 'trip', 'stay_days', 'start_date', 'company', 'dep_time_local', 'stop_info', 'duration']


from pyspark.sql.window import Window

Threshold = 5

w =  Window.partitionBy(byVar).orderBy('search_date').rowsBetween(0, sys.maxsize)
flight6 = (flight5.withColumn('future_min_price', F.min(col('price')).over(w))
          .withColumn('price_will_drop',                       
                      (col('price') - col('future_min_price')) > Threshold ))
flight6 = flight6.withColumn('price_will_drop_num', flight6.price_will_drop.cast('int'))
flight6.cache()
# min_price = (flight5
#                   .groupBy(byVar)
#                   .agg(F.min(col("price"))).alias("min_price"))

# flight6 = flight5.join(min_price, on=byVar, how='left')
# # flight6.select('price', 'min(price)').show(10)
# # flight6.select('price', 'min(price)').show()
# flight6 = flight6.withColumn('price_diff', col('price')-col('min(price)'))
# flight6.show()

DataFrame[airline_code: string, airline_codes: array<string>, arr_time: timestamp, check_bag_not_inc: boolean, company: string, dep_time: timestamp, duration: string, flight_code: string, flight_number: string, from_city_name: string, id: string, index: bigint, plane: string, power: boolean, price: double, price_code: string, search_date: timestamp, span_days: int, start_date: date, stay_days: int, stop: int, stop_info: string, table_name: string, task_id: string, ticket_left: string, to_city_name: string, trip: string, version: string, video: boolean, wifi: boolean, dep_time_local: timestamp, arr_time_local: timestamp, duration_h: string, duration_m: string, stop_info1: string, stop_info2: string, stop_loc1: string, stop_duration1: string, stop_loc2: string, stop_duration2: string, stop_duration_h1: string, stop_duration_m1: string, stop_duration_h2: string, stop_duration_m2: string, duration_minutes: int, stop1_minutes: int, stop2_minutes: int, arr_time_group: string, dep_time_group:

In [219]:
flight6.filter((flight6.trip==1) & \
               (flight6.start_date=='2017-10-01') & \
               (flight6.company=='AirAsiaX')) \
    .select('duration', 'search_date', 'price', 'future_min_price', 'price_will_drop') \
    .show(100)

+--------+--------------------+-----+----------------+---------------+
|duration|         search_date|price|future_min_price|price_will_drop|
+--------+--------------------+-----+----------------+---------------+
|  14h20m|2017-04-04 00:00:...|529.5|           439.0|           true|
|  14h20m|2017-04-06 00:00:...|529.5|           439.0|           true|
|  14h20m|2017-04-15 00:00:...|529.5|           439.0|           true|
|  14h20m|2017-04-16 00:00:...|529.5|           439.0|           true|
|  14h20m|2017-04-19 00:00:...|529.5|           439.0|           true|
|  14h20m|2017-04-20 00:00:...|529.5|           439.0|           true|
|  14h20m|2017-04-21 00:00:...|579.5|           439.0|           true|
|  14h20m|2017-04-22 00:00:...|579.5|           439.0|           true|
|  14h20m|2017-04-24 00:00:...|579.5|           439.0|           true|
|  14h20m|2017-04-25 00:00:...|579.5|           439.0|           true|
|  14h20m|2017-04-26 00:00:...|535.5|           439.0|           true|
|  14h

In [214]:
flight2.unpersist()

DataFrame[airline_code: string, airline_codes: array<string>, arr_time: timestamp, check_bag_not_inc: boolean, company: string, dep_time: timestamp, duration: string, flight_code: string, flight_number: string, from_city_name: string, id: string, index: bigint, plane: string, power: boolean, price: double, price_code: string, search_date: timestamp, span_days: int, start_date: date, stay_days: int, stop: int, stop_info: string, table_name: string, task_id: string, ticket_left: string, to_city_name: string, trip: string, version: string, video: boolean, wifi: boolean, dep_time_local: timestamp, arr_time_local: timestamp, duration_h: string, duration_m: string, stop_info1: string, stop_info2: string, stop_loc1: string, stop_duration1: string, stop_loc2: string, stop_duration2: string, stop_duration_h1: string, stop_duration_m1: string, stop_duration_h2: string, stop_duration_m2: string, duration_minutes: int, stop1_minutes: int, stop2_minutes: int, arr_time_group: string, dep_time_group:

In [216]:
flight6.cache()

DataFrame[airline_code: string, airline_codes: array<string>, arr_time: timestamp, check_bag_not_inc: boolean, company: string, dep_time: timestamp, duration: string, flight_code: string, flight_number: string, from_city_name: string, id: string, index: bigint, plane: string, power: boolean, price: double, price_code: string, search_date: timestamp, span_days: int, start_date: date, stay_days: int, stop: int, stop_info: string, table_name: string, task_id: string, ticket_left: string, to_city_name: string, trip: string, version: string, video: boolean, wifi: boolean, dep_time_local: timestamp, arr_time_local: timestamp, duration_h: string, duration_m: string, stop_info1: string, stop_info2: string, stop_loc1: string, stop_duration1: string, stop_loc2: string, stop_duration2: string, stop_duration_h1: string, stop_duration_m1: string, stop_duration_h2: string, stop_duration_m2: string, duration_minutes: int, stop1_minutes: int, stop2_minutes: int, arr_time_group: string, dep_time_group:

In [217]:
# ['table_name', 'trip', 'stay_days', 'start_date', 'company', 'dep_time_local', 'stop_info', 'duration']
# groupby(flight2.trip, flight2.stay_days, flight.dep_time_group).agg(func.mean('price')).show()
# flight6.groupBy(col('trip'), col('stay_days'), col('lead_time')).agg(F.mean('price_will_drop_num')).show()
flight6.groupBy(col('trip'), col('stay_days'), col('lead_time')). \
    agg(F.mean('price_will_drop_num')).orderBy(['trip', 'stay_days', 'lead_time'], ascending=[1, 1, 0]).show(1000)


+----+---------+---------+------------------------+
|trip|stay_days|lead_time|avg(price_will_drop_num)|
+----+---------+---------+------------------------+
|   1|        0|      180|     0.36288265306122447|
|   1|        0|      179|     0.35714285714285715|
|   1|        0|      178|      0.3497304582210243|
|   1|        0|      177|     0.37575757575757573|
|   1|        0|      176|     0.33737244897959184|
|   1|        0|      175|     0.32705248990578734|
|   1|        0|      174|      0.3657289002557545|
|   1|        0|      173|       0.361376673040153|
|   1|        0|      172|     0.36607142857142855|
|   1|        0|      171|     0.36630504148053605|
|   1|        0|      170|      0.3647959183673469|
|   1|        0|      169|     0.37333333333333335|
|   1|        0|      168|      0.3832908163265306|
|   1|        0|      167|      0.3860880663688577|
|   1|        0|      166|     0.38049713193116635|
|   1|        0|      165|                 0.40625|
|   1|      

# Save to parquet

In [221]:
flight6.coalesce(2).write.parquet("C:\\s3\\20170503_jsonl\\flight6.parquet")

In [9]:
# flight6 = spark.read.parquet("C:\\s3\\20170503_jsonl\\flight6.parquet")
flight6 = spark.read.parquet("/home/ubuntu/parquet/flight6.parquet")


In [10]:
flight6.count()

1835752

In [11]:
flight6.printSchema()

root
 |-- airline_code: string (nullable = true)
 |-- airline_codes: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- arr_time: timestamp (nullable = true)
 |-- check_bag_not_inc: boolean (nullable = true)
 |-- company: string (nullable = true)
 |-- dep_time: timestamp (nullable = true)
 |-- duration: string (nullable = true)
 |-- flight_code: string (nullable = true)
 |-- flight_number: string (nullable = true)
 |-- from_city_name: string (nullable = true)
 |-- id: string (nullable = true)
 |-- index: long (nullable = true)
 |-- plane: string (nullable = true)
 |-- power: boolean (nullable = true)
 |-- price: double (nullable = true)
 |-- price_code: string (nullable = true)
 |-- search_date: timestamp (nullable = true)
 |-- span_days: integer (nullable = true)
 |-- start_date: date (nullable = true)
 |-- stay_days: integer (nullable = true)
 |-- stop: integer (nullable = true)
 |-- stop_info: string (nullable = true)
 |-- table_name: string (nullable = tru

In [12]:
flight7 = (flight6.withColumn('price_drop_amt', col('price') - col('future_min_price'))
                  .withColumn('stop_minutes', col('stop1_minutes') + col('stop2_minutes'))
                  .withColumn('few_tickets_left', col('ticket_left') != '0')
                  .select('price_will_drop_num', 'price', 'few_tickets_left', 'from_city_name', 'to_city_name', 'trip', 'stay_days',
                          'company', 'flight_code', 'plane', 'span_days', 'stop',
                         'power', 'video', 'wifi', 'check_bag_not_inc', 
                         'dep_time_group', 'arr_time_group',  
                         'lead_time', 'dep_weekday', 'dep_weeknum', 
                         'days_to_last_holiday', 'days_to_next_holiday',                         
                         'duration_minutes', 'stop_minutes'
                      ).filter(flight6.price > 0))
flight7.dtypes
flight7 = flight7.na.fill({'stop_minutes': 0, 'days_to_next_holiday': 999})
flight7.show(2, truncate=False)

+-------------------+-------+----------------+--------------+------------+----+---------+--------------+-----------+-------------------------+---------+----+-----+-----+-----+-----------------+--------------+--------------+---------+-----------+-----------+--------------------+--------------------+----------------+------------+
|price_will_drop_num|price  |few_tickets_left|from_city_name|to_city_name|trip|stay_days|company       |flight_code|plane                    |span_days|stop|power|video|wifi |check_bag_not_inc|dep_time_group|arr_time_group|lead_time|dep_weekday|dep_weeknum|days_to_last_holiday|days_to_next_holiday|duration_minutes|stop_minutes|
+-------------------+-------+----------------+--------------+------------+----+---------+--------------+-----------+-------------------------+---------+----+-----+-----+-----+-----------------+--------------+--------------+---------+-----------+-----------+--------------------+--------------------+----------------+------------+
|1        

How to transform categorical variables? See
http://stackoverflow.com/questions/32982425/encode-and-assemble-multiple-features-in-pyspark


# One hot encoding

In [13]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder

column_vec_in = ['from_city_name', 'to_city_name', 'trip', 'company', 'flight_code', 'plane', 
                 'arr_time_group', 'dep_time_group', 'dep_weekday']
column_vec_out = ['from_city_name_catVec','to_city_name_catVec', 'trip_catVec',
                'company_catVec', 'flight_code_catVec', 'plane_catVec', 
                  'arr_time_group_catVec', 'dep_time_group_catVec', 'dep_weekday_catVec']
 
indexers = [StringIndexer(inputCol=x, outputCol=x+'_tmp') for x in column_vec_in ]
 
encoders = [OneHotEncoder(dropLast=False, inputCol=x+"_tmp", outputCol=y)
            for x,y in zip(column_vec_in, column_vec_out)]

tmp = [[i,j] for i,j in zip(indexers, encoders)]
tmp = [i for sublist in tmp for i in sublist]


# Prepare features and labels

In [14]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler, SQLTransformer

# prepare labeled sets
# 'price_will_drop',

cols_now = ['price', 'few_tickets_left', 'stay_days',
              'span_days', 'stop',
             'power', 'video', 'wifi', 'check_bag_not_inc',                          
             'lead_time', 'dep_weeknum', 
             'days_to_last_holiday', 'days_to_next_holiday',                         
             'duration_minutes', 'stop_minutes',
            'from_city_name_catVec','to_city_name_catVec', 'trip_catVec',
            'company_catVec', 'flight_code_catVec', 'plane_catVec', 
            'arr_time_group_catVec', 'dep_time_group_catVec', 'dep_weekday_catVec']

assembler_features = VectorAssembler(inputCols=cols_now, outputCol='features')
# labelIndexer = StringIndexer(inputCol='price', outputCol="label")
# tmp += [assembler_features, labelIndexer]
tmp += [assembler_features]
pipeline = Pipeline(stages=tmp)

In [15]:
# flight3.na.drop().count()

# from pyspark.ml import Pipeline
# from pyspark.ml.feature import StringIndexer

# indexers = [StringIndexer(inputCol=column, outputCol=column+"_index")\
#             .fit(flight3) for column in list(set(flight3.columns)-\
#                                              set(['stay_days', 'power', 'price', 'span_days',\
#                                                   'stop', 'video', 'wifi', 'duration_minutes',\
#                                                  'dep_weeknum', 'lead_time']))]

# pipeline = Pipeline(stages=indexers)
# flight4 = pipeline.fit(flight3).transform(flight3)

# flight4.show()

In [16]:
cols = ['from_city_name', 'to_city_name', 'trip', 'company', 'flight_code', 'plane', 
                 'arr_time_group', 'dep_time_group', 'dep_weekday']

for c in cols:
    flight7.groupBy(col(c)).count().show(50000)
    
    
flight7 = flight7.na.replace('', 'unknown', 'plane')

# http://stackoverflow.com/questions/40711229/pyspark-ml-error-urequirement-failed-cannot-have-an-empty-string-for-name
# replacements = {
#   'plane': 'unknown', 'another_col': 'another_replacement',
#   'numeric_column_wont_be_replaced': 1.0
# }

# for k, v in replacements.items():
#     # We can replace string only if target is string
#     # In Python 2 str -> basestring
#     if isinstance(v, str):
#         df = df.na.replace("", v, [k])        

+--------------+-------+
|from_city_name|  count|
+--------------+-------+
|        sydney|1835752|
+--------------+-------+

+------------+-------+
|to_city_name|  count|
+------------+-------+
|    shanghai|1835752|
+------------+-------+

+----+-------+
|trip|  count|
+----+-------+
|   1| 295267|
|   2|1540485|
+----+-------+

+--------------------+------+
|             company| count|
+--------------------+------+
|      Cathay Pacific|482702|
| Philippine Airlines|   714|
|    Juneyao Airlines|   701|
|      Japan Airlines| 41421|
|    Virgin Australia| 80045|
|China Southern Ai...|230960|
|     Hainan Airlines| 46185|
|Thai Airways Inte...| 52266|
|        Fiji Airways| 38559|
|   Malaysia Airlines| 47129|
|    Vietnam Airlines| 21673|
|     Asiana Airlines| 30163|
|      China Airlines|  8390|
|          Air Canada|    62|
|           Air China|134287|
|  All Nippon Airways| 42670|
|  Singapore Airlines| 72912|
|            AirAsiaX| 66254|
|    Sichuan Airlines| 25138|
|LATAM 

In [17]:
cols = ['price_will_drop', 'price', 'few_tickets_left', 'from_city_name', 'to_city_name', 'trip', 'stay_days',
                          'company', 'flight_code', 'plane', 'span_days', 'stop',
                         'power', 'video', 'wifi', 'check_bag_not_inc', 
                         'dep_time_group', 'arr_time_group',  
                         'lead_time', 'dep_weekday', 'dep_weeknum', 
                         'days_to_last_holiday', 'days_to_next_holiday',                         
                         'duration_minutes', 'stop_minutes']

for c in cols:
    display(c + " :" + str(flight7.where(col(c).isNull()).count()))

'price_will_drop :0'

'price :0'

'few_tickets_left :0'

'from_city_name :0'

'to_city_name :0'

...

'dep_weeknum :0'

'days_to_last_holiday :0'

'days_to_next_holiday :0'

'duration_minutes :0'

'stop_minutes :0'

In [18]:
flight7.show(2)
flight7.count()

+-------------------+-------+----------------+--------------+------------+----+---------+--------------+-----------+--------------------+---------+----+-----+-----+-----+-----------------+--------------+--------------+---------+-----------+-----------+--------------------+--------------------+----------------+------------+
|price_will_drop_num|  price|few_tickets_left|from_city_name|to_city_name|trip|stay_days|       company|flight_code|               plane|span_days|stop|power|video| wifi|check_bag_not_inc|dep_time_group|arr_time_group|lead_time|dep_weekday|dep_weeknum|days_to_last_holiday|days_to_next_holiday|duration_minutes|stop_minutes|
+-------------------+-------+----------------+--------------+------------+----+---------+--------------+-----------+--------------------+---------+----+-----+-----+-----+-----------------+--------------+--------------+---------+-----------+-----------+--------------------+--------------------+----------------+------------+
|                  1|1671

1835752

In [20]:
# flight7 = flight7.na.drop()
output = pipeline.fit(flight7).transform(flight7)
output = output.withColumnRenamed('price_will_drop_num', 'label')
output.cache()

DataFrame[label: int, price: double, few_tickets_left: boolean, from_city_name: string, to_city_name: string, trip: string, stay_days: int, company: string, flight_code: string, plane: string, span_days: int, stop: int, power: boolean, video: boolean, wifi: boolean, check_bag_not_inc: boolean, dep_time_group: string, arr_time_group: string, lead_time: int, dep_weekday: string, dep_weeknum: int, days_to_last_holiday: int, days_to_next_holiday: int, duration_minutes: int, stop_minutes: int, from_city_name_tmp: double, from_city_name_catVec: vector, to_city_name_tmp: double, to_city_name_catVec: vector, trip_tmp: double, trip_catVec: vector, company_tmp: double, company_catVec: vector, flight_code_tmp: double, flight_code_catVec: vector, plane_tmp: double, plane_catVec: vector, arr_time_group_tmp: double, arr_time_group_catVec: vector, dep_time_group_tmp: double, dep_time_group_catVec: vector, dep_weekday_tmp: double, dep_weekday_catVec: vector, features: vector]

In [21]:
output.show(2)

Py4JError: An error occurred while calling o648.showString

In [ ]:
# flight4.columns

In [ ]:
# flight4.select('stay_days',
#  'power', 
#  'span_days',
#  'stop',
#  'video',
#  'wifi',
#  'duration_minutes',
#  'dep_weeknum',
#  'lead_time',
#  'trip_index',
#  'arr_time_group_index',
#  'to_city_name_index',
#  'ticket_left_index',
#  'company_index',
#  'from_city_name_index',
#  'airline_code_index',
#  'plane_index',
#  'check_bag_not_inc_index',
#  'dep_weekday_index',
#  'dep_time_group_index').dtypes

In [ ]:
# from pyspark.ml.linalg import Vectors
# from pyspark.ml.feature import VectorAssembler

# # from pyspark.ml.feature import VectorAssembler

# # ignore = ['id', 'label', 'binomial_label']
# # assembler = VectorAssembler(
# #     inputCols=[x for x in df.columns if x not in ignore],
# #     outputCol='features')

# # assembler.transform(df)
# # arr_time_group

# assembler = VectorAssembler(    
#     inputCols=[
#  'stay_days',
#  'power', 
#  'span_days',
#  'stop',
#  'video',
#  'wifi',
#  'duration_minutes', 
#  'dep_weeknum',
#  'lead_time',
#  'trip_index',
#  'arr_time_group_index',
#  'to_city_name_index',
#  'ticket_left_index',
#  'company_index',
#  'from_city_name_index',
#  'airline_code_index',
#  'plane_index',
#  'check_bag_not_inc_index',
#  'dep_weekday_index',
#  'dep_time_group_index'],
#     outputCol="features")

# output = assembler.transform(flight4)
# # print("Assembled columns 'hour', 'mobile', 'userFeatures' to vector column 'features'")
# # output.select("features", "clicked").show(truncate=False)
# output = output.withColumnRenamed('price', 'label')
# output.show(5, truncate=False)

In [22]:
display(output.count(), flight6.count(), flight7.count())

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:40455)

In [12]:
# output.coalesce(2).write.parquet("C:\\s3\\20170503_jsonl\\output.parquet")
output.coalesce(2).write.parquet("/home/ubuntu/parquet/output.parquet/output.parquet")

In [6]:
output = spark.read.parquet("/home/ubuntu/parquet/output.parquet/output.parquet")

In [12]:
output.count()

18379

In [11]:
output = output.sample(False, 0.1, seed=0)

In [13]:
output.count()
output.printSchema()
output.show(2)

root
 |-- label: integer (nullable = true)
 |-- price: double (nullable = true)
 |-- few_tickets_left: boolean (nullable = true)
 |-- from_city_name: string (nullable = true)
 |-- to_city_name: string (nullable = true)
 |-- trip: string (nullable = true)
 |-- stay_days: integer (nullable = true)
 |-- company: string (nullable = true)
 |-- flight_code: string (nullable = true)
 |-- plane: string (nullable = true)
 |-- span_days: integer (nullable = true)
 |-- stop: integer (nullable = true)
 |-- power: boolean (nullable = true)
 |-- video: boolean (nullable = true)
 |-- wifi: boolean (nullable = true)
 |-- check_bag_not_inc: boolean (nullable = true)
 |-- dep_time_group: string (nullable = true)
 |-- arr_time_group: string (nullable = true)
 |-- lead_time: integer (nullable = true)
 |-- dep_weekday: string (nullable = true)
 |-- dep_weeknum: integer (nullable = true)
 |-- days_to_last_holiday: integer (nullable = true)
 |-- days_to_next_holiday: integer (nullable = true)
 |-- duration_m

# interlude to model in h2o (not sparkling water)  
When converting the whole dataset to pandas dataframe, there was not enough RAM, JAVA was forced to shutdown!!!

In [14]:
output.printSchema()
output2 = output.select('label', 'features')
# output2 = output.select('label', 'features').rdd

root
 |-- label: integer (nullable = true)
 |-- price: double (nullable = true)
 |-- few_tickets_left: boolean (nullable = true)
 |-- from_city_name: string (nullable = true)
 |-- to_city_name: string (nullable = true)
 |-- trip: string (nullable = true)
 |-- stay_days: integer (nullable = true)
 |-- company: string (nullable = true)
 |-- flight_code: string (nullable = true)
 |-- plane: string (nullable = true)
 |-- span_days: integer (nullable = true)
 |-- stop: integer (nullable = true)
 |-- power: boolean (nullable = true)
 |-- video: boolean (nullable = true)
 |-- wifi: boolean (nullable = true)
 |-- check_bag_not_inc: boolean (nullable = true)
 |-- dep_time_group: string (nullable = true)
 |-- arr_time_group: string (nullable = true)
 |-- lead_time: integer (nullable = true)
 |-- dep_weekday: string (nullable = true)
 |-- dep_weeknum: integer (nullable = true)
 |-- days_to_last_holiday: integer (nullable = true)
 |-- days_to_next_holiday: integer (nullable = true)
 |-- duration_m

In [15]:
output2.count()

18379

In [18]:
# output2.count()
flight_pd = output.select('label', 'price', 'few_tickets_left', 
                          'from_city_name', 'to_city_name', 'trip', 'stay_days',
                          'company', 'flight_code', 'plane', 'span_days', 'stop',
                         'power', 'video', 'wifi', 'check_bag_not_inc', 
                         'dep_time_group', 'arr_time_group',  
                         'lead_time', 'dep_weekday', 'dep_weeknum', 
                         'days_to_last_holiday', 'days_to_next_holiday',                         
                         'duration_minutes', 'stop_minutes').toPandas()
flight_pd.head()

,label,price,few_tickets_left,from_city_name,to_city_name,trip,stay_days,company,flight_code,plane,...,check_bag_not_inc,dep_time_group,arr_time_group,lead_time,dep_weekday,dep_weeknum,days_to_last_holiday,days_to_next_holiday,duration_minutes,stop_minutes
0,0,689.52,True,sydney,shanghai,1,0,China Southern Airlines,CZ302,Airbus A330,...,False,evening,afternoon,47,5,23,-11,113,1060,0
1,1,1025.42,True,sydney,shanghai,1,0,Malaysia Airlines,MH140,AIRBUS INDUSTRIE A330-300,...,False,evening,afternoon,64,3,26,-30,94,1085,0
2,1,1283.27,True,sydney,shanghai,1,0,Qantas Airways,QF508,BOEING 737-800 (WINGLETS) PASSENGER,...,False,morning,evening,71,3,27,-37,87,860,0
3,0,723.82,True,sydney,shanghai,1,0,Cathay Pacific,CX138,BOEING 777-300ER,...,False,evening,evening,91,2,28,-43,81,1540,0
4,0,663.82,True,sydney,shanghai,1,0,China Eastern Airlines,MU8440,AIRBUS INDUSTRIE A330-300,...,False,morning,evening,84,0,29,-48,76,655,0


In [19]:
# flight_pd.to_csv('C:\\s3\\20170503_jsonl\\flight_pd.csv', sep='\t')
flight_pd.to_csv('/home/ubuntu/parquet/flight_pd.csv', sep='\t')

In [20]:
h2o.init()
# from h2o.estimators.gbm import H2OGradientBoostingEstimator

# hf = h2o.H2OFrame(flight_pd)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_131"; OpenJDK Runtime Environment (build 1.8.0_131-8u131-b11-0ubuntu1.16.04.2-b11); OpenJDK 64-Bit Server VM (build 25.131-b11, mixed mode)
  Starting server from /home/ubuntu/anaconda3/lib/python3.5/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp2iu8aa2f
  JVM stdout: /tmp/tmp2iu8aa2f/h2o_ubuntu_started_from_python.out
  JVM stderr: /tmp/tmp2iu8aa2f/h2o_ubuntu_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,05 secs
H2O cluster version:,3.10.4.6
H2O cluster version age:,16 days
H2O cluster name:,H2O_from_python_ubuntu_x1de28
H2O cluster total nodes:,1
H2O cluster free memory:,232.5 Mb
H2O cluster total cores:,1
H2O cluster allowed cores:,1
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


In [21]:
import pandas as pd
flight_pd=pd.read_csv("/home/ubuntu/parquet/flight_pd.csv", delimiter='\t')

In [22]:
# flight_pd.head()
# help(pd.read_csv)
flight_hex = h2o.H2OFrame(flight_pd)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [23]:
train, valid, test = flight_hex.split_frame([0.6, 0.2], seed=1234)

flight_X = flight_hex.col_names[2:]
flight_y = flight_hex.col_names[1]    


In [24]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator

rf_v1 = H2ORandomForestEstimator(
    model_id="rf_v1",
    ntrees=200,
    stopping_rounds=2,
    score_each_iteration=True,
    seed=1000000)

rf_v1.train(flight_X, flight_y, training_frame=train, validation_frame=valid)
rf_v1
rf_v1.score_history()
model_path = h2o.save_model(model=rf_v1, path="C:\\s3\\20170503_jsonl", force=True)
print(model_path)

drf Model Build progress: |███████████████████████████████████████████████| 100%
/home/ubuntu/.jupyter/flight_project/C:\s3\20170503_jsonl/rf_v1


In [25]:
rf_v1

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  rf_v1


ModelMetricsRegression: drf
** Reported on train data. **

MSE: 0.16125699307615135
RMSE: 0.4015681674089112
MAE: 0.29576985113399795
RMSLE: 0.2812286384019287
Mean Residual Deviance: 0.16125699307615135

ModelMetricsRegression: drf
** Reported on validation data. **

MSE: 0.14772317434168744
RMSE: 0.3843477258182848
MAE: 0.2896960866913029
RMSLE: 0.26993483527383644
Mean Residual Deviance: 0.14772317434168744
Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
,2017-05-13 12:52:38,0.052 sec,0.0,nan,nan,nan,nan,nan,nan
,2017-05-13 12:52:39,0.773 sec,1.0,0.5375521,0.2919111,0.2889622,0.5433976,0.2976626,0.2952809
,2017-05-13 12:52:39,1.137 sec,2.0,0.5221194,0.2976923,0.2726086,0.4712592,0.2990894,0.2220852
,2017-05-13 12:52:39,1.453 sec,3.0,0.5050030,0.2936024,0.2550280,0.4416678,0.2929564,0.1950704
,2017-05-13 12:52:39,1.713 sec,4.0,0.4906770,0.2913865,0.2407639,0.4279878,0.2926597,0.1831736
---,---,---,---,---,---,---,---,---,---
,2017-05-13 12:52:44,6.563 sec,27.0,0.4036201,0.2946506,0.1629092,0.3850148,0.2889101,0.1482364
,2017-05-13 12:52:45,6.746 sec,28.0,0.4029845,0.2951376,0.1623965,0.3846213,0.2890857,0.1479335
,2017-05-13 12:52:45,6.912 sec,29.0,0.4023413,0.2951724,0.1618785,0.3843001,0.2891586,0.1476865
,2017-05-13 12:52:45,7.214 sec,30.0,0.4023116,0.2960082,0.1618546,0.3844270,0.2898464,0.1477841



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
price,8528.5175781,1.0,0.1670183
flight_code,8061.0830078,0.9451916,0.1578643
lead_time,6366.9272461,0.7465456,0.1246868
days_to_last_holiday,3866.3369141,0.4533422,0.0757164
few_tickets_left,3569.9750977,0.4185927,0.0699126
dep_weeknum,3275.1225586,0.3840201,0.0641384
stay_days,3227.5683594,0.3784442,0.0632071
dep_weekday,2826.2167969,0.3313843,0.0553472
duration_minutes,2584.9619141,0.3030963,0.0506226
days_to_next_holiday,2132.4052734,0.2500323,0.0417600


In [26]:
# output2.take(2)
(trainingData, testData) = output2.randomSplit([0.7, 0.3])

# Stopped here! 20170513

In [35]:
from time import time
from pyspark.ml.classification import LogisticRegression

# Load training data
# training = spark.read.format("libsvm").load("data/mllib/sample_libsvm_data.txt")

lr = LogisticRegression(maxIter=100, regParam=0.1, elasticNetParam=0.8)

# Fit the model
t0 = time()
lrModel = lr.fit(trainingData)
tt = time() - t0

# Print the coefficients and intercept for logistic regression
print("Classifier trained in " + str(tt) + " seconds.")
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

Classifier trained in 117.26345801353455 seconds.
Coefficients: (415,[1,17,18],[-0.636888486254,0.02752806023,-0.0236746971456])
Intercept: -0.15746604930887775


In [36]:
from pyspark.ml.classification import LogisticRegression

# Extract the summary from the returned LogisticRegressionModel instance trained
# in the earlier example
trainingSummary = lrModel.summary

# Obtain the objective per iteration
objectiveHistory = trainingSummary.objectiveHistory
print("objectiveHistory:")
for objective in objectiveHistory:
    print(objective)

# Obtain the receiver-operating characteristic as a dataframe and areaUnderROC.
trainingSummary.roc.show()
print("areaUnderROC: " + str(trainingSummary.areaUnderROC))

# Set the model threshold to maximize F-Measure
fMeasure = trainingSummary.fMeasureByThreshold
maxFMeasure = fMeasure.groupBy().max('F-Measure').select('max(F-Measure)').head()
bestThreshold = fMeasure.where(fMeasure['F-Measure'] == maxFMeasure['max(F-Measure)']) \
    .select('threshold').head()['threshold']
lr.setThreshold(bestThreshold)

objectiveHistory:
0.65965578316531
0.65711778074993
0.6526917948919334
0.6500999301284001
0.6492599933704122
0.6489641671631153
0.6485216921434988
0.6484842065443268
0.6483063221990527
0.6482723762111214
0.6482516218060789
0.6482254209192646
0.6481976474933386
0.6481650074625309
0.6481069533543712
0.6477526451025746
0.6475130339686226
0.6474578323572088
0.647396090473081
0.6473465984142018
0.6472936896040737
0.6472569602439301
0.6472170722058855
0.6471895139254257
0.6471591800353315
0.6471434501387139
0.6470705812041906
0.6470686210102691
0.6470669959768485
0.6470618925480771
0.647060755627432
0.6470597286031927
0.6470580906753532
0.6470578826597967
0.64705783197502
0.6470577567273563
0.6470576866332658
0.6470576435581784
0.6470576237233439
0.6470575811617785
0.6470575110695101
0.6470572963566015
0.6470572552241214
0.6470571417411584
0.6470571010875171
0.647057036177969
0.6470570151283103
0.6470569585685809
0.6470568210794492
0.6470567704462217
0.6470567477581171
0.6470566341047528
0.6

LogisticRegression_4eefbed75b8eca6922ce

In [39]:
# Make predictions.
predictions = lrModel.transform(testData)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)
predictions.select('label', 'prediction').coalesce(1).write.csv('D://Data Science//pySpark//check_pred7.csv')

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|    0|(415,[0,1,2,4,5,6...|
|       0.0|    0|(415,[0,1,2,4,5,6...|
|       0.0|    0|(415,[0,1,2,4,5,6...|
|       0.0|    0|(415,[0,1,2,4,5,6...|
|       0.0|    0|(415,[0,1,2,4,5,6...|
+----------+-----+--------------------+
only showing top 5 rows



Py4JJavaError: An error occurred while calling o884.csv.
: java.lang.UnsupportedOperationException: CSV data source does not support struct<type:tinyint,size:int,indices:array<int>,values:array<double>> data type.
	at org.apache.spark.sql.execution.datasources.csv.CSVFileFormat.org$apache$spark$sql$execution$datasources$csv$CSVFileFormat$$verifyType$1(CSVFileFormat.scala:233)
	at org.apache.spark.sql.execution.datasources.csv.CSVFileFormat$$anonfun$verifySchema$1.apply(CSVFileFormat.scala:237)
	at org.apache.spark.sql.execution.datasources.csv.CSVFileFormat$$anonfun$verifySchema$1.apply(CSVFileFormat.scala:237)
	at scala.collection.Iterator$class.foreach(Iterator.scala:893)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1336)
	at scala.collection.IterableLike$class.foreach(IterableLike.scala:72)
	at org.apache.spark.sql.types.StructType.foreach(StructType.scala:96)
	at org.apache.spark.sql.execution.datasources.csv.CSVFileFormat.verifySchema(CSVFileFormat.scala:23

In [ ]:
from pyspark.ml import Pipeline
# from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import VectorIndexer
# from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Load and parse the data file, converting it to a DataFrame.
# data = spark.read.format("libsvm").load("data/mllib/sample_libsvm_data.txt")

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=100).fit(output)
    
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = output.randomSplit([0.7, 0.3])

# Train a RandomForest model.
# rf = RandomForestRegressor(featuresCol="indexedFeatures", numTrees=1000, featureSubsetStrategy="auto",
#                                     impurity='variance', maxDepth=4, maxBins=32)
gbt = GBTClassifier(labelCol="label", featuresCol="indexedFeatures", maxIter=10)

# Chain indexer and forest in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, gbt])

# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
# evaluator = RegressionEvaluator(
#     labelCol="label", predictionCol="prediction", metricName="rmse")
evaluator = BinaryClassificationEvaluator(
    labelCol="label", predictionCol="prediction")
auc = evaluator.evaluate(predictions)
print("Area under the curve (AUC) on test data = %g" % auc)

gbtModel = model.stages[1]
print(gbtModel)  # summary only

In [ ]:
from pyspark.ml.classification import LogisticRegression

# Load training data
# training = spark.read.format("libsvm").load("data/mllib/sample_libsvm_data.txt")
flight7.withColumnRenamed('price_will_drop_num', 'label')

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(training)

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

how to score model manually: http://stackoverflow.com/questions/35731140/apply-model-scores-to-spark-dataframe-python

In [44]:
predictions.select('label', 'prediction').coalesce(1).write.csv('D://Data Science//pySpark//check_pred6.csv')

AnalysisException: 'path file:/D:/Data Science/pySpark/check_pred6.csv already exists.;'

In [ ]:
predictions.show(20)

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Load and parse the data file, converting it to a DataFrame.
data = flight3

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
# featureIndexer =\
#     VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)


# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a GBT model.
gbt = GBTRegressor(featuresCol="indexedFeatures", maxIter=10)

# Chain indexer and GBT in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, gbt])

# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

gbtModel = model.stages[1]
print(gbtModel)  # summary only



In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# # Load and parse the data file, converting it to a DataFrame.
# data = spark.read.format("libsvm").load("data/mllib/sample_libsvm_data.txt")

# # Automatically identify categorical features, and index them.
# # Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(flight3)

# # Split the data into training and test sets (30% held out for testing)
# (trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a GBT model.
gbt = GBTRegressor(featuresCol="indexedFeatures", maxIter=10)

# Chain indexer and GBT in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, gbt])

# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

gbtModel = model.stages[1]
print(gbtModel)  # summary only

In [ ]:
# display(flight.select("duration").show())
# display(flight2.select("duration_h", 'duration_m').show())
display(flight2.show(5))
# flight2.toPandas().head()

In [ ]:
import re
def getHours(x):
  return re.match('([0-9]+(?=h))', x)
temp = flight.select("duration").rdd.map(lambda x:getHours(x[0])).toDF()
temp.select("duration").show()

In [ ]:
flight.select("arr_time").show(2, truncate=False)

In [ ]:
# flight2 = (flight.withColumn('arr_time_local', flight.arr_time.substr(1, 23).cast('date'))                 
#           )

# flight3 = flight2.withColumn('arr_time', flight2.arr_time.cast('timestamp'))

# .select('arr_time').show(2, truncate=False)


# flight2 = (flight.withColumn('duration_minutes', flight.duration.substr(1, 23).cast('date'))                 
#           )

# flight.selectExpr("duration", "regexp_extract(duration,'([0-9]+(?=h))', 1) as duration_hour", "regexp_extract(duration,'([0-9]+(?=m))', 1)", ).show()
flight.selectExpr("regexp_extract(duration,'([0-9]+(?=h))', 1) as duration_hour").duration_hour.cast("double").show()

In [ ]:
flight2.select('arr_time', 'arr_time_local').dtypes
flight2.select('arr_time', 'arr_time_local').show(2, truncate=False)

In [ ]:
# from pyspark.sql import crosstab
# from pyspark.sql.functions import *

flight.crosstab('start_date','from_city_name').show()
# flight.columns


3
down vote
In order to simply cast a string column to a timestamp, the string column must be properly formatted.

To retrieve the "createdAt" column as a timestamp, you can write the UDF function that would convert the string

"2016-07-01T16:37:41-0400"
to

"2016-07-01 16:37:41"
and convert the "createdAt" column to a new format (don't forget to handle the timezone field).

Once you have a column containing timestamps as strings like "2016-07-01 16:37:41", a simple cast to timestamp would do the job, as you have it in your code.

You can read more about Date/Time/String Handling in Spark here.

In [ ]:
display(flight.select("arr_time").take(3))
display(flight2.select("arr_time").take(3))



In [ ]:
flight2.withColumn('local', F.from_utc_timestamp(flight2.arr_time, "AEST")).select("local").show()
flight2.withColumn('local', F.from_utc_timestamp(flight2.arr_time, "CTZ")).select("local").show()

In [ ]:
import time
from datetime import datetime

import pytz # $ pip install pytz
from tzlocal import get_localzone # $ pip install tzlocal



In [ ]:
# get local timezone    
local_tz = get_localzone() 
print(local_tz)
    

In [ ]:
# test it
# utc_now, now = datetime.utcnow(), datetime.now()
ts = time.time()
utc_now, now = datetime.utcfromtimestamp(ts), datetime.fromtimestamp(ts)
print(utc_now)
        

In [ ]:
local_now = utc_now.replace(tzinfo=pytz.utc).astimezone(local_tz) # utc -> local
print(local_now)


In [ ]:
assert local_now.replace(tzinfo=None) == now

In [ ]:
# test it
# utc_now, now = datetime.utcnow(), datetime.now()
ts = time.time()
utc_now, now = datetime.utcfromtimestamp(ts), datetime.fromtimestamp(ts)

local_now = utc_now.replace(tzinfo=pytz.utc).astimezone(local_tz) # utc -> local
assert local_now.replace(tzinfo=None) == now


* Spark-Sql doesn't support date-time, and nor timezones
* Using timestamp is the only solution
* from_unixtime(at) parses the epoch time correctly, just that the printing of it as a string changes it due to timezone. It is safe to assume that the  from_unixtime will convert it correctly ( although printing it might show different results)
* from_utc_timestamp will shift ( not just convert) the timestamp to that timezone, in this case it will subtract 8 hours to the time since (-08:00)
* printing sql results messes up the times with respect to timezone param
  	 	
* from_unixtime(at) does what from_utc_timestamp does too, it will parse a Unix timestamp integer (seconds since midnight 1970-01-01), and convert the time instant parsed from UTC to the system's default timezone. 


In [ ]:
flight2.selectExpr("from_utc_timestamp(arr_time, 'AEST') as testthis").show()

In [ ]:
flight2.dtypes
display(flight2.select('arr_time').take(3))
display(flight.select('arr_time').take(3))


In [ ]:
from pyspark.sql.functions import year, month, dayofmonth, hour, minute, weekofyear, crosstab

flight2.select(year("arr_time").alias('year'), 
               month("arr_time").alias('month'),
               dayofmonth("arr_time").alias('day'),
               hour("arr_time").alias('hour'),
               minute("arr_time").alias('minute'),
               weekofyear('arr_time').alias('week_no'),
               "arr_time_zone"
              ).show()



In [ ]:
from pyspark.sql.functions import regexp_extract, col

#flight.select(regexp_extract('arr_time', r'[+-][0-9]{2}:[0-9]{2}\b', 1)).alias('d').collect






In [ ]:
df = spark.createDataFrame([('2017-04-09T07:15:00.000+08:00',)], ['str'])
df.select(regexp_extract('str', '(\d+)-(\d+)', 1).alias('d')).collect()

In [ ]:
flight2.describe('price').show()

In [ ]:
from pyspark.sql.functions import *
# flight2.dtypes
# flight2.first()
display(flight2.select(max("start_date")).show())
display(flight2.select(min("start_date")).show())

In [ ]:
flight2 = flight2.withColumn('zero_price', flight2.)

In [ ]:
# flight.describe().show()
flight.describe("from_city_name").show() 

In [ ]:
from pyspark.sql.functions import colflight.filter(flight.price > 0).groupby(flight.search_date_x, flight.from_city_name, flight.to_city_name).count().show()

In [ ]:
from pyspark.sql.functions import col
# flight_to_brisbane = flight.where(col("price") > 0 & col("to_city_name") == "brisbane").groupby(flight.search_date_x).count()

In [ ]:
df.where(col("v").isin({"foo", "bar"})).count()

In [ ]:
numeric = sqlContext.createDataFrame([
    ('3.5,', '5.0', 'null'), ('2.0', '14.0', 'null'),  ('null', '38.0', 'null'),
    ('null', 'null', 'null'),  ('1.0', 'null', '4.0')],
    ('low', 'high', 'normal'))

numeric_filtered_1 = numeric.where(numeric['LOW'] != 'null')
numeric_filtered_1.show()

In [ ]:
flight.select('from_city_name', 'price').show(5)

h2o tutorial 
https://github.com/h2oai/h2o-tutorials/blob/master/tutorials/pysparkling/Chicago_Crime_Demo.md